In [24]:
import torch
import pandas as pd
import sklearn.datasets as datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from collections import Counter

In [53]:
class Node():
    def __init__(self, feature=None, threshold=None, data_left=None, data_right=None, gain=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.data_left = data_left
        self.data_right = data_right
        self.gain = gain
        self.value = value

class DecisionTree():
    def __init__(self, min_sample = 2, max_depth = 5, impurity_function=None):
        self.min_sample = min_sample
        self.max_depth = max_depth
        self.impurity_function = impurity_function
        self.root = None
    
    def _entropy(self, y):
        _, counts = torch.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return -(probs * torch.log2(probs)).sum()
    
    def _information_gain(self, y, y_left, y_right):
        p = y_left.shape[0] / y.shape[0]
        return self._entropy(y) - p * self._entropy(y_left) - (1 - p) * self._entropy(y_right)
    
    def _gini(self, y):
        _, counts = torch.unique(y, return_counts=True)
        probs = counts / counts.sum()
        return 1 - (probs ** 2).sum()
    
    def _gini_gain(self, y, y_left, y_right):
        p = y_left.shape[0] / y.shape[0]
        return self._gini(y) - p * self._gini(y_left) - (1 - p) * self._gini(y_right)

    def _calc_gain(self, y, y_left, y_right):
        if self.impurity_function == "gini":
            return self._gini_gain(y, y_left, y_right)
        elif self.impurity_function == "entropy":
            return self._information_gain(y, y_left, y_right)
    
    def _best_split(self, X, y):
        best_gain = -1
        best_split = {}
        
        for feature in range(X.shape[1]):
            X_crr = X[:, feature]

            for threshold in torch.unique(X_crr):
                df = torch.concat((X, y.reshape(1, -1).T), dim=1)
                df_left = df[df[:, feature] <= threshold]
                df_right = df[df[:, feature] > threshold]
                if len(df_left) > 0 and len(df_right) > 0:
                    y = df[:, -1]
                    gain = self._calc_gain(y, df_left[:, -1], df_right[:, -1])
                    if gain > best_gain:
                        best_gain = gain
                        best_split = {"feature": feature, 
                                    "threshold": threshold, 
                                    "data_left": df_left, 
                                    "data_right": df_right,
                                    "gain": gain}
        return best_split

    def _build(self, X, y, depth = 0):
        if X.shape[0] >= self.min_sample and depth <= self.max_depth:
            best = self._best_split(X, y)

            if best['gain'] > 0:
                left = self._build(best["data_left"][:, :-1], best["data_left"][:, -1], depth + 1)
                right = self._build(best["data_right"][:, :-1], best["data_right"][:, -1], depth + 1)
                return Node(feature=best["feature"], 
                            threshold=best["threshold"], 
                            data_left=left, 
                            data_right=right, 
                            gain = best["gain"])
        return Node(
            value=Counter(y).most_common(1)[0][0]
        )
    
    def fit(self, X, y):
        X = torch.tensor(X)
        y = torch.tensor(y)
        self.root = self._build(X, y)
    
    def _predict(self, X, tree):
        # return leaf value if we are at a leaf node
        if tree.value is not None:
            return tree.value
        # traverse the tree
        feature = tree.feature
        threshold = tree.threshold
        if X[feature] < threshold:
            # go left
            return self._predict(X, tree.data_left)
        # go right
        return self._predict(X, tree.data_right)
    
    def predict(self, X):
        return torch.tensor([self._predict(x, self.root) for x in X])     
    
    def printTree(self, tree, level=0):
        if tree.value is not None:
            print("|-" * level + "Predict", tree.value.item())
        else:
            print("|-" * level + "X[{}] <= {}".format(tree.feature, tree.threshold))
            self.printTree(tree.data_left, level + 1)
            self.printTree(tree.data_right, level + 1)

# Iris data

In [5]:

iris = datasets.load_iris()
iris_df = pd.DataFrame(iris.data, columns = ["sepal length (cm)", "sepal width (cm)", "petal length (cm)", "petal width (cm)"])
iris_df['target'] = iris.target
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2
146,6.3,2.5,5.0,1.9,2
147,6.5,3.0,5.2,2.0,2
148,6.2,3.4,5.4,2.3,2


In [6]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.2, random_state=42)
print(X_train.shape)
print(X_test.shape)

(120, 4)
(30, 4)


## Impurity function - Entropy

In [54]:
DecisionTreeModel = DecisionTree(min_sample= 2, max_depth= 5, impurity_function= "entropy")
DecisionTreeModel.fit(X_train, y_train)
y_pred = DecisionTreeModel.predict(X_test)
y_pred

tensor([1., 0., 2., 1., 1., 0., 1., 2., 1., 1., 2., 0., 0., 0., 0., 1., 2., 1.,
        1., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 0.], dtype=torch.float64)

In [55]:
DecisionTreeModel.printTree(DecisionTreeModel.root)

X[2] <= 1.9
|-Predict 0.0
|-X[2] <= 4.7
|-|-X[3] <= 1.6
|-|-|-Predict 1.0
|-|-|-Predict 2.0
|-|-X[3] <= 1.7
|-|-|-X[2] <= 4.9
|-|-|-|-Predict 1.0
|-|-|-|-X[3] <= 1.5
|-|-|-|-|-Predict 2.0
|-|-|-|-|-X[0] <= 6.7
|-|-|-|-|-|-Predict 1.0
|-|-|-|-|-|-Predict 2.0
|-|-|-X[2] <= 4.8
|-|-|-|-X[0] <= 5.9
|-|-|-|-|-Predict 1.0
|-|-|-|-|-Predict 2.0
|-|-|-|-Predict 2.0


In [29]:
accuracy_score(y_test, y_pred)

1.0

## Impurity function - Gini

In [31]:
DecisionTreeModel = DecisionTree(min_sample= 2, max_depth= 5, impurity_function= "gini")
DecisionTreeModel.fit(X_train, y_train)
y_pred = DecisionTreeModel.predict(X_test)
y_pred

tensor([1., 0., 2., 1., 1., 0., 1., 2., 1., 1., 2., 0., 0., 0., 0., 1., 2., 1.,
        1., 2., 0., 2., 0., 2., 2., 2., 2., 2., 0., 0.], dtype=torch.float64)

In [32]:
accuracy_score(y_test, y_pred)

1.0